# Starbucks Store Information Scraping

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd

In [29]:
# webdriver로 browser 준비
browser = webdriver.Chrome()
browser.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

### CSS selector로 위치를 찾아서 클릭하기
 browser에서 모든 정보가 로딩된 뒤에 click해야 제대로 작동하기 때문에 sleep을 설정하는 것이 좋다. 그러나 Jupyter notebook에서는 cell이 하나씩 순차적으로 실행되니까 크게 상관이 없긴 하다. <br>
 또는 전체를 run하는 경우, 너무 빨리 지나가버리기 때문에 sleep을 설정하면 진행을 천천히 확인할 수 있다.

In [4]:
import time
time.sleep(5)
browser.find_element_by_css_selector('ul.sido_arae_box > li  > a[data-sidocd="01"]').click()    # '서울' 클릭

In [5]:
time.sleep(5)
browser.find_element_by_css_selector('ul.gugun_arae_box > li > a[href*="0"][data-guguncd=""]').click()   # '전체' 클릭

위 CSS selector 중 `[href*="0"]` 부분은 selector를 특정하는데 도움이 되지 않아서 필요 없음. 단지 tag attribute를 여러개 연이어 사용할 수 있다는 것을 보여주기 위해서 추가함

### BeautifulSoup으로 매장 리스트를 읽어오기

In [9]:
page = browser.page_source            # 현재 webdriver에 열려있는 페이지의 내용을 불러옴
soup = BS(page, 'html.parser')

In [11]:
tags = soup.select('li.quickResultLstCon')
len(tags), tags[0]                        # 총 554개 매장

(554,
 <li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>)

In [12]:
# 매장이름, 주소, 위도, 경도, 매장타입을 스크래핑
starbucks = []

for tag in tags:
    name = tag['data-name']
    add = tag.select_one('p').text
    lat = tag['data-lat']
    long = tag['data-long']
    store = tag.select_one('i').text
    starbucks.append([name, add, lat, long, store])

starbucks[:5]

[['역삼아레나빌딩',
  '서울특별시 강남구 언주로 425 (역삼동)1522-3232',
  '37.501087',
  '127.043069',
  '리저브 매장 2번'],
 ['논현역사거리',
  '서울특별시 강남구 강남대로 538 (논현동)1522-3232',
  '37.510178',
  '127.022223',
  '리저브 매장 2번'],
 ['신사역성일빌딩',
  '서울특별시 강남구 강남대로 584 (논현동)1522-3232',
  '37.514132',
  '127.020563',
  '리저브 매장 2번'],
 ['국기원사거리',
  '서울특별시 강남구 테헤란로 125 (역삼동)1522-3232',
  '37.499517',
  '127.031495',
  '리저브 매장 2번'],
 ['스탈릿대치R',
  '서울특별시 강남구 남부순환로 2947 (대치동)1522-3232',
  '37.494668',
  '127.062583',
  '리저브 매장 2번']]

### DataFrame으로 만들기

In [13]:
pd.DataFrame(starbucks, columns=['Store', 'Address', 'Latitude', 'Longitude', 'Type']).head()

,Store,Address,Latitude,Longitude,Type
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동)1522-3232,37.501087,127.043069,리저브 매장 2번
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동)1522-3232,37.510178,127.022223,리저브 매장 2번
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동)1522-3232,37.514132,127.020563,리저브 매장 2번
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동)1522-3232,37.499517,127.031495,리저브 매장 2번
4,스탈릿대치R,서울특별시 강남구 남부순환로 2947 (대치동)1522-3232,37.494668,127.062583,리저브 매장 2번


---
### 번외) WebElement object의 html source 확인하는 법
Selenium으로 element 찾기를 마친 뒤 BeautifulSoup으로 parsing 할 수도 있다!

In [14]:
# outerHTML: 지정한 태그(li) 포함해서 가져옴
# innerHTML: 지정한 태그 제외 안쪽만 가져옴
elmt = browser.find_element_by_css_selector('li.quickResultLstCon').get_attribute('outerHTML')
elmt, type(elmt)                  # str

('<li class="quickResultLstCon" style="background:#fff" data-lat="37.501087" data-long="127.043069" data-index="0" data-name="역삼아레나빌딩" data-code="3762" data-storecd="1509" data-hlytag="null">\t<strong data-store="1509" data-yn="N" data-name="역삼아레나빌딩" data-my_siren_order_store_yn="N">역삼아레나빌딩  </strong>\t<p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br>1522-3232</p>\t<i class="pin_general">리저브 매장 2번</i></li>',
 str)

In [15]:
print(BS(elmt, 'html.parser').prettify())

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff">
 <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">
  역삼아레나빌딩
 </strong>
 <p class="result_details">
  서울특별시 강남구 언주로 425 (역삼동)
  <br/>
  1522-3232
 </p>
 <i class="pin_general">
  리저브 매장 2번
 </i>
</li>


### Selenium만 사용해서 Starbucks 매장정보 가져오기

In [139]:
# webdriver로 browser 준비
browser = webdriver.Chrome()
browser.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

In [125]:
import time
time.sleep(5)
browser.find_element_by_css_selector('ul.sido_arae_box > li  > a[data-sidocd="01"]').click()    # '서울' 클릭

In [126]:
time.sleep(5)
browser.find_element_by_css_selector('ul.gugun_arae_box > li > a[href*="0"][data-guguncd=""]').click()   # '전체' 클릭

In [127]:
# 매장정보 모두 가져오기
time.sleep(3)
tags = browser.find_elements_by_css_selector('li.quickResultLstCon')
tags[0].get_attribute('outerHTML')          # 첫번째 WebElement의 html source 확인

'<li class="quickResultLstCon" style="background:#fff" data-lat="37.501087" data-long="127.043069" data-index="0" data-name="역삼아레나빌딩" data-code="3762" data-storecd="1509" data-hlytag="null">\t<strong data-store="1509" data-yn="N" data-name="역삼아레나빌딩" data-my_siren_order_store_yn="N">역삼아레나빌딩  </strong>\t<p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br>1522-3232</p>\t<i class="pin_general">리저브 매장 2번</i></li>'

#### Scrolling을 통해서 text를 불러오지 못하는 문제 해결
아래 tag.find_element를 수행할 때, 화면에 보이는 내용(.is_displayed() == True)에 대한 text만 가져오는 문제를 해결하기 위해서 scroll down 이용 
* **ActionChains()**: 여기서 발생하는 무한루프의 문제가 해결 안됨. `Break`, `reset_actions()`, `Keys.ESCAPE`, `또다른 ActionChains()`를 만들어 멈추게 하는것 등 모두 효과를 보지 못함
* **.location_once_scrolled_into_view**: 아래서 사용한 방법으로 오류 없음!

In [128]:
# Selenium은 렌더링 하느라 속도가 느림...
stores = []

for tag in tags:
    # scroll을 통해서 일부 text를 가져오지 못하는 문제 해결
    tag.location_once_scrolled_into_view                 # WebElement로 이동
    
    name = tag.get_attribute('data-name')    
    add = tag.find_element_by_tag_name('p').text.replace('\n', ' ') 
    lat = tag.get_attribute('data-lat')
    long = tag.get_attribute('data-long')
    store = tag.find_element_by_tag_name('i').text       # 위에서 스크롤 다운했기 때문에 문제 없음
    stores.append([name, add, lat, long, store])

In [134]:
stores[:5]

[['역삼아레나빌딩',
  '서울특별시 강남구 언주로 425 (역삼동) 1522-3232',
  '37.501087',
  '127.043069',
  '리저브 매장 2번'],
 ['논현역사거리',
  '서울특별시 강남구 강남대로 538 (논현동) 1522-3232',
  '37.510178',
  '127.022223',
  '리저브 매장 2번'],
 ['신사역성일빌딩',
  '서울특별시 강남구 강남대로 584 (논현동) 1522-3232',
  '37.514132',
  '127.020563',
  '리저브 매장 2번'],
 ['국기원사거리',
  '서울특별시 강남구 테헤란로 125 (역삼동) 1522-3232',
  '37.499517',
  '127.031495',
  '리저브 매장 2번'],
 ['스탈릿대치R',
  '서울특별시 강남구 남부순환로 2947 (대치동) 1522-3232',
  '37.494668',
  '127.062583',
  '리저브 매장 2번']]

In [135]:
# Dataframe으로 만들기
pd.DataFrame(stores, columns=['Store', 'Address', 'Latitude', 'Longitude', 'Type']).head(15)

,Store,Address,Latitude,Longitude,Type
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동) 1522-3232,37.501087,127.043069,리저브 매장 2번
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동) 1522-3232,37.510178,127.022223,리저브 매장 2번
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동) 1522-3232,37.514132,127.020563,리저브 매장 2번
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동) 1522-3232,37.499517,127.031495,리저브 매장 2번
4,스탈릿대치R,서울특별시 강남구 남부순환로 2947 (대치동) 1522-3232,37.494668,127.062583,리저브 매장 2번
5,봉은사역,서울특별시 강남구 봉은사로 619 (삼성동) 1522-3232,37.515000,127.063196,리저브 매장 2번
6,압구정윤성빌딩,서울특별시 강남구 논현로 834 (신사동) 1522-3232,37.5227934,127.0286009,리저브 매장 2번
7,코엑스별마당,서울특별시 강남구 영동대로 513 (삼성동) 1522-3232,37.510150,127.060275,리저브 매장 2번
8,삼성역섬유센터R,서울특별시 강남구 테헤란로 518 (대치동) 1522-3232,37.507750,127.060651,리저브 매장 2번
9,압구정R,서울특별시 강남구 언주로 861 (신사동) 1522-3232,37.5273669,127.033061,리저브 매장 2번


In [140]:
# 브라우저 끄기
browser.quit()